### Sprawdzić prawo Little'a

E(x) = lambdaA * E(R)

gdzie
- E(x) - Średnia liczba zadań w systemie
- E(R) - Średni czas spędzony w systemie
- lambdaA - tempo przybywania zadań

In [2]:
import numpy as np

In [3]:
lambdaA = 1/20
lambdaS_values = [1/15, 1/100, 1/5]

In [4]:
def generate_time(l):
    return -np.log(1 - np.random.uniform(0, 1)) / l

In [5]:
def simulate_process(limit, l_s):
    time = 0
    queue = []
    completed_tasks = 0
    total_task_time_in_system = 0
    report_tasks_in_queue = 0
    report_count = 0

    task_arrival_time = generate_time(lambdaA)
    task_dequeue_time = task_arrival_time
    task_completion_time = task_dequeue_time + 1 # fake value
    while time <= limit:
        if time == task_arrival_time:
            queue.append(time)
            task_arrival_time += generate_time(lambdaA)

        if time == task_completion_time:
            completed_tasks += 1

        if time == task_dequeue_time:
            arrival_time = queue.pop(0)
            task_completion_time = time + generate_time(l_s)
            total_task_time_in_system += task_completion_time - arrival_time

            if len(queue) > 0:
                task_dequeue_time = task_completion_time
            else:
                task_dequeue_time = max(task_completion_time, task_arrival_time)

        report_tasks_in_queue += len(queue)
        report_count += 1
        time = min(task_arrival_time, task_dequeue_time, task_completion_time) if time != task_completion_time else min(task_arrival_time, task_dequeue_time)

    return report_tasks_in_queue / report_count, total_task_time_in_system / completed_tasks

In [6]:
simulations = 1_000
time_limit = 10_000

for l_s in lambdaS_values:
  report_ER = []
  report_EX = []
  for _ in range(simulations):
    ex, er = simulate_process(time_limit, l_s)
    report_EX.append(ex)
    report_ER.append(er)

  ex = np.mean(report_EX)
  er = np.mean(report_ER)

  ex_exp = np.abs(ex - (er * lambdaA))

  print(f'Dla lambdaS = {l_s}: E(R) = {er}, E(R) * lambdaA = {er * lambdaA}, E(X) = {ex},  delta = {ex_exp - ex} %delta = {np.abs(ex - ex_exp) / ex}\n')

Dla lambdaS = 0.06666666666666667: E(R) = 59.11531793177099, E(R) * lambdaA = 2.9557658965885496, E(X) = 2.576300886909984,  delta = -2.1968358772314183 %delta = 0.8527093587528528

Dla lambdaS = 0.01: E(R) = 4087.0528488332725, E(R) * lambdaA = 204.35264244166365, E(X) = 199.61935529078974,  delta = -194.88606813991584 %delta = 0.9762884358384044

Dla lambdaS = 0.2: E(R) = 6.663527675883941, E(R) * lambdaA = 0.33317638379419706, E(X) = 0.20846193012677708,  delta = -0.0837474764593571 %delta = 0.4017399071783787

